# Python 3

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import numpy as np
import networkx as nx
import scipy.sparse as sp
import pickle
import time
from itertools import combinations, chain
from joblib import Parallel, delayed
import datetime as dt

In [5]:
n_genes = 500
n_drugs = 400
n_secombo = 3
# Creates random network as graph
start = time.time()
gene_net = nx.planted_partition_graph(50, 10, 0.15, 0.05, seed=42)
ppi_adj = nx.adjacency_matrix(gene_net)
print('PPI adjacency matrix created in ',dt.timedelta(seconds=time.time()-start))
ppi_degrees = np.array(ppi_adj.sum(axis=0)).squeeze()

PPI adjacency matrix created in  0:00:00.047678


In [6]:
# Creates random adjacency matrix for genes and drugs
start = time.time()
dti_adj = sp.csr_matrix((10 * np.random.randn(n_genes, n_drugs) > 15).astype(int))
print('DTI adjacency matrix created in ',dt.timedelta(seconds=time.time()-start))

DTI adjacency matrix created in  0:00:00.049444


In [8]:
# DDI adjacency matrices
start = time.time()
tmp = np.dot(dti_adj.T, dti_adj)
def se_adj_matrix(i):
    mat = np.zeros([n_drugs,n_drugs],dtype=int)
    for d1, d2 in combinations(list(range(n_drugs)), 2):
        if tmp[d1, d2] == i + 5:
            mat[d1, d2] = mat[d2, d1] = 1
    return sp.csr_matrix(mat) 
ddi_adj_list = Parallel(n_jobs=8)\
    (delayed(se_adj_matrix)(d) for d in range(n_secombo))        
ddi_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in ddi_adj_list]
print('DDI adjacency matrices created in ',dt.timedelta(seconds=time.time()-start))

DDI adjacency matrices created in  0:00:02.189292


In [9]:
# Protein Features
prot_feat = sp.identity(n_genes)
# Drug Features
drug_feat = sp.identity(n_drugs)

# Saving

In [10]:
filename = './data_structures/DS_toy_genes'+str(n_genes)+'_drugs'+str(n_drugs)+'_se'\
+str(n_secombo)
data = {}
# Dictionaries
# DDI
data['ddi_adj_list'] = ddi_adj_list
data['ddi_degrees_list'] = ddi_degrees_list
# DTI
data['dti_adj'] = dti_adj
# PPI
data['ppi_adj'] = ppi_adj
data['ppi_degrees'] = ppi_degrees
# DSE
data['drug_feat'] = drug_feat
# PF
data['prot_feat'] = prot_feat

In [11]:
with open(filename, 'wb') as f:
    pickle.dump(data, f, protocol=2)